In [53]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from collections import defaultdict, Counter
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, fbeta_score
#from sklearn.ensemble import AdaBoostClassifier

from sklearn.datasets import make_classification


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import RandomOverSampler

import scipy.stats as stats


In [2]:
#Read the dataset
df_acc_all = pd.read_csv('/Users/dona/Documents/DonaRay/DataIncubator/project/Acc.csv')
df_veh_all = pd.read_csv('/Users/dona/Documents/DonaRay/DataIncubator/project/Veh.csv')
df_cas_all = pd.read_csv('/Users/dona/Documents/DonaRay/DataIncubator/project/Cas.csv')


df_cas_freq = df_cas_all.drop(['Age_of_Casualty'], axis=1)
df_cas_classFreq = df_cas_all[['Age_of_Casualty']]




/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Drop columns/rows with missing/nan values
df_acc_sub = df_acc_all.drop(['LSOA_of_Accident_Location', 'Location_Northing_OSGR', 'Location_Easting_OSGR', 'Longitude', 'Latitude'], axis=1)
df_acc = df_acc_sub.dropna()

for date, time in zip(df_acc.Date, df_acc.Time):
  date_obj = datetime.strptime(date, '%d/%m/%Y')
  date = date_obj
  time_obj = datetime.strptime(time, '%H:%M')
  time = time_obj

In [4]:
# Merge Data sets

#Merge Vehicle and Causalty data sets: 
df_veh_cas = pd.merge(df_veh_all, df_cas_all, on=['Accident_Index', 'Vehicle_Reference'], how='left')
df_all = pd.merge(df_acc, df_veh_cas, on='Accident_Index')
df_acc_veh = pd.merge(df_acc, df_veh_all, on='Accident_Index')

print(df_acc_veh.head())
print(df_all.head(20))

  Accident_Index  Police_Force  Accident_Severity  Number_of_Vehicles  \
0  2016010000005             1                  3                   2   
1  2016010000005             1                  3                   2   
2  2016010000006             1                  3                   1   
3  2016010000008             1                  3                   1   
4  2016010000016             1                  3                   1   

   Number_of_Casualties        Date  Day_of_Week   Time  \
0                     1  01/11/2016            3  02:30   
1                     1  01/11/2016            3  02:30   
2                     1  01/11/2016            3  00:37   
3                     1  01/11/2016            3  01:25   
4                     1  01/11/2016            3  09:15   

   Local_Authority_(District) Local_Authority_(Highway)         ...          \
0                          28                 E09000005         ...           
1                          28                 E0

In [56]:
# Define the target variable

# Number of data points
n_records = df_acc_veh['Accident_Severity'].size
print(n_records)

Target_Sev = pd.Series(99, np.arange(n_records))
Urban = pd.Series(0, np.arange(n_records))
Rural = pd.Series(0, np.arange(n_records))
Male = pd.Series(0, np.arange(n_records))
Female = pd.Series(0, np.arange(n_records))

j = 0
for _, row in df_acc_veh.iterrows():
    if ((row['Accident_Severity'] == 1) or (row['Accident_Severity'] == 2)):
        Target_Sev[j] = 1
    else:
        Target_Sev[j] = 0
        
            # Urban and Rural
    if (row['Urban_or_Rural_Area'] == 1): # Urban
        Urban[j] = 1
    elif (row['Urban_or_Rural_Area'] == 2): # Rural
        Rural[j] = 1
    else:
        Urban[j] = 0
    
    # Define Male, Female dummy variables
    if (row['Sex_of_Driver'] == 1): # Male
        Male[j] = 1
    elif (row['Sex_of_Driver'] == 2): # Rural
        Female[j] = 1
    else:
        Male[j] = 0
    
        
    j += 1
       
df = pd.concat([Target_Sev, Male, Urban], axis =1)


print(df.head())

252431
   0  1
0  0  0
1  0  0
2  0  0
3  0  0
4  0  0


In [57]:
print(type(df))
df.columns = ['Target_Sev', 'Male', 'Urban']
df['Speed'] = df_acc_veh['Speed_limit']
print(df.head())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
   Target_Sev  Male  Urban  Speed
0           0     1      1   30.0
1           0     1      1   30.0
2           0     0      1   30.0
3           0     1      1   30.0
4           0     1      1   30.0
          Target_Sev          Male          Urban          Speed
count  252431.000000  252431.00000  252431.000000  252431.000000
mean        0.159992       0.64895       0.637037      38.560201
std         0.366600       0.47730       0.480855      14.378074
min         0.000000       0.00000       0.000000      20.000000
25%         0.000000       0.00000       0.000000      30.000000
50%         0.000000       1.00000       1.000000      30.000000
75%         0.000000       1.00000       1.000000      50.000000
max         1.000000       1.00000       1.000000      70.000000


In [62]:
# Frequentist approach: Hypothesis test and Confidence Interval

#df = pd.concat([Target_Sev, Male], axis =1)

# Adding header to the dataframe df
#df.columns = ['Target_Sev', 'Male']

df_m = df[df['Urban'] == 1]
df_f = df[df['Urban'] == 0]

n_m = len(df_m['Target_Sev'])
n_f = len(df_f['Target_Sev'])

# Compute sample proportions: p_w, p_b
p_m = np.sum(df_m['Target_Sev'] == 1) / n_m
p_f = np.sum(df_f['Target_Sev'] == 1) / n_f

# Compute pooled "p" under the null hypothesis that Ho: p_m = p_f
p_bar = (n_m * p_m + n_f * p_f) / (n_m + n_f)

# Compute sample standard deviation of (p_w - p_b): std_p_wb
std_p_mf = np.sqrt(p_bar * (1-p_bar) * (1/n_m + 1/n_f))

# Compute the z test statistic: z
z = (p_m - p_f) / std_p_mf

print("z value: {:.4f}".format(z))

# Compute 95% Confidence rates by Frequentist method
# Compute Standard error of the difference in proportions
std_diff_prop = np.sqrt((p_m * (1-p_m)/n_m) + (p_f * (1-p_f)/n_f))

CI_low = (p_m - p_f) - 1.96 * std_diff_prop
CI_high = (p_m - p_f) + 1.96 * std_diff_prop

print("Standard error of differnce in proportions is: {:.4f}" .format(std_diff_prop))
print("Confidence Interval of difference in proportions is: [{:.4f}, {:.4f}]" .format(CI_low, CI_high))

z value: -49.0814
Standard error of differnce in proportions is: 0.0016
Confidence Interval of difference in proportions is: [-0.0776, -0.0714]


In [60]:
# Chi-square test to test indepenced of categorical data with more than two categories
# Test independence of target variable and Speed limit


# Crosstab

target_speed_table = pd.crosstab(df['Target_Sev'], df['Speed'])

#print(type(target_speed_table))
#print(target_speed_table.columns)
#print(target_speed_table)

Slight = target_speed_table.iloc[0].tolist()
Severe = target_speed_table.iloc[1].tolist()

x_value = np.array([Slight, Severe])
print(type(Slight))

#print(x_value)


chi_square = stats.chi2_contingency(x_value)
print(chi_square)




<class 'list'>
(3246.266508318586, 0.0, 5, array([[  9588.68863174, 128612.74881453,  18856.49429745,
          9699.56965666,  28299.02157817,  16987.47702144],
       [  1826.31136826,  24496.25118547,   3591.50570255,
          1847.43034334,   5389.97842183,   3235.52297856]]))
